In [22]:
!pip install statsmodels --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from sqlalchemy import create_engine
import numpy as np
import matplotlib
from time import time
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta, date, datetime
import matplotlib.ticker as tkr
from scipy import stats
plt.style.use('fivethirtyeight')
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
matplotlib.rcParams['text.color'] = 'k'
from pylab import rcParams
from statsmodels.tsa.seasonal import STL

%matplotlib inline

In [24]:
from google.colab import files
import io
uploaded = files.upload()

Saving Global DataBase3.csv to Global DataBase3 (1).csv
Saving Global DataBase2 daily.csv to Global DataBase2 daily (1).csv


In [34]:
df1=pd.read_csv(io.BytesIO(uploaded['Global DataBase2 daily.csv']), index_col='datetime_per_day', parse_dates=True, dayfirst=True)

df2=pd.read_csv(io.BytesIO(uploaded['Global DataBase3.csv']), index_col='datetime', parse_dates=True, dayfirst=True)


df1 = df1.loc['2017-01-01':'2022-02-28']
df2 = df2.loc['2017-01-01':'2022-02-28']

index = pd.date_range(start='2017-01-01', end='2022-02-28', freq="1d")
columns = ["datetime_per_month", 'Price - PT [€/MWh]','Price - ES [€/MWh]','Actual Consumption (MWh)','Biomass AA','Fossil Gas AA','Fossil Hard Coal AC', 'Hydro Pumped Storage AA','Hydro Pumped Storage AC','Hydro Run-of-river and poundage AA','Hydro Water Reservoir AA', 'Other AA', 'Solar AA','Wind Onshore AA','Electricity Consumption (MWh)']

df = pd.DataFrame(np.nan, index=range(0,index.size), columns=columns)
df["datetime_per_month"] = index
df.set_index("datetime_per_month", inplace=True)

df = df.resample("M").mean()


columns1 = ['Electricity Consumption (MWh)']
for column in columns1 : 
    df[column] = df2[column].resample("M").sum()
columns2=['Price - PT [€/MWh]','Price - ES [€/MWh]','Actual Consumption (MWh)','Biomass AA','Fossil Gas AA','Fossil Hard Coal AC', 'Hydro Pumped Storage AA','Hydro Pumped Storage AC','Hydro Run-of-river and poundage AA','Hydro Water Reservoir AA', 'Other AA', 'Solar AA','Wind Onshore AA']
for column in columns2 : 
    df[column] = df2[column].resample("M").mean()
df = df.sort_index()

#Handeling global outliers
GlobalOutliers =  []
for i in range(13):
    GlobalOutliers.append(df.idxmax()['Price - PT [€/MWh]'])
    df.at[df.idxmax()['Price - PT [€/MWh]'], 'Price - PT [€/MWh]'] = np.nan

#Replacing them with the mean of that day
for d in GlobalOutliers:
    sdt = datetime.combine(d, datetime.min.time())
    edt = sdt + timedelta(hours=24)
    df.at[d, 'Price - PT [€/MWh]'] = df[sdt:edt]['Price - PT [€/MWh]'].mean()
    
#Handling negative values 
negativeIndex = df.index[df['Price - PT [€/MWh]'] < 0]
for index in (negativeIndex):
    df.at[index, 'Price - PT [€/MWh]']= df.at[index - timedelta(hours=1), 'Price - PT [€/MWh]']
    
df = df.dropna()


In [35]:
# Split into train and test set
data = df[: '2021-11-01 00:00:00']
test_data = df['2021-11-01 00:00:00': ]

In [36]:
start_time = time()

fitted_model1 = ExponentialSmoothing(data["Price - PT [€/MWh]"],trend='add').fit()

exec_time1 = time()-start_time

In [29]:
test_predictions1 = fitted_model1.forecast(test_data.shape[0])

ValueError: ignored

In [30]:
start_time = time()

fitted_model2 = ExponentialSmoothing(data['Price - PT [€/MWh]'],trend='add',seasonal='add',seasonal_periods=12).fit()

exec_time2 = time()-start_time

In [31]:
test_predictions2 = fitted_model2.forecast(test_data.shape[0])

ValueError: ignored

In [32]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions1)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions1)


print('*************** Exponential Smoothing Results ***************')
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time1)

ValueError: ignored

In [ ]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions2)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions2)


print('*************** Holt Winter Results ***************')
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time2)

In [ ]:
R2